In [28]:
import os
import random
from sklearn.model_selection import train_test_split

# Путь к директории, содержащей трёх папки
dataset_dir = 'dataset/'
import torch.nn.functional as F
import torch
import gc

gc.collect()

torch.cuda.empty_cache()

torch.cuda.memory_summary(device=None, abbreviated=False)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
image_paths = []

# Итерируемся по каждой папке и получаем пути к изображениям
for root, dirs, files in os.walk(dataset_dir):
    for file in files:
        if file.endswith('.jpeg') or file.endswith('.png') or file.endswith('.jpg'):
            image_paths.append(os.path.join(root, file))
train_data = []
test_data = []
train_paths, test_paths = train_test_split(image_paths, test_size=0.2, random_state=42)
for path in train_paths:
    train_data.append(path)

# Добавляем пути к тестовым данным
for path in test_paths:
    test_data.append(path)

# Проверяем, что пути к данным были правильно разделены
print('Количество тренировочных изображений:', len(train_data))
print('Количество тестовых изображений:', len(test_data))

Количество тренировочных изображений: 44873
Количество тестовых изображений: 11219


In [29]:
import torch
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

In [30]:
def load_dataset():
    data_transform = transforms.Compose([
        transforms.Resize((64, 64)),
        transforms.ToTensor()])
    
    dataset = datasets.ImageFolder(root='dataset', transform=data_transform)
    train_size = int(0.8 * len(dataset))
    test_size = len(dataset) - train_size
    train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])
    
    # Создаем загрузчики данных
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=16, shuffle=True)
    
    return train_loader, test_loader

In [31]:
import torch.nn as nn
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, 3)
        self.fc1 = nn.Linear(32 *4 * 49, 120)  # Update the input size of fc1
        self.fc2 = nn.Linear(120, 84)
        #self.fc3 = nn.Linear(84, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.size(0), -1) 
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        #x = self.fc3(x)
        return x

In [32]:
def train(model, train_loader, criterion, optimizer, epoch, writer):
    model.train()
    
    for i, (inputs, labels) in enumerate(train_loader, 0):
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        
        loss = criterion(outputs, labels)
        
        loss.backward()
        
        optimizer.step()
        
        if i % 100 == 99:
            writer.add_scalar('training loss', loss.item(), epoch * len(train_loader) + i)
            print('Epoch: %d, Batch: %5d, Loss: %.3f' %
                  (epoch, i + 1, loss.item()))

In [33]:
def test(model, test_loader, criterion, epoch, writer):
    model.eval()
    
    test_loss = 0
    correct = 0
    
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            outputs = model(inputs)
            
            test_loss += criterion(outputs, labels).item()
            
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).sum().item()
    
    test_loss /= len(test_loader.dataset)
    
    accuracy = 100.0 * correct / len(test_loader.dataset)
    
    print('Test set: Average loss: %.4f, Accuracy: %.2f%%' % (test_loss, accuracy))
    
    writer.add_scalar('test loss', test_loss, epoch)
    writer.add_scalar('accuracy', accuracy, epoch)

In [34]:
from torch.utils.tensorboard import SummaryWriter

In [35]:
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter

def main():
    # Инициализация модели, функции потерь и оптимизатора
    model = CNN()
    if torch.cuda.is_available():
        model.cuda()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
    
    # Создание загрузчиков данных
    train_loader, test_loader = load_dataset()
    
    # Подключение tensorboard
    writer = SummaryWriter()
    
    # Обучение и тестирование модели
    for epoch in range(10):
        train(model, train_loader, criterion, optimizer, epoch, writer)
        test(model, test_loader, criterion, epoch, writer)
    
    writer.close()

if __name__ == '__main__':
    main()

Epoch: 0, Batch:   100, Loss: 2.763
Epoch: 0, Batch:   200, Loss: 3.994
Epoch: 0, Batch:   300, Loss: 3.461
Epoch: 0, Batch:   400, Loss: 3.597
Epoch: 0, Batch:   500, Loss: 3.720
Epoch: 0, Batch:   600, Loss: 2.781
Epoch: 0, Batch:   700, Loss: 3.643
Epoch: 0, Batch:   800, Loss: 3.741
Epoch: 0, Batch:   900, Loss: 3.064
Epoch: 0, Batch:  1000, Loss: 2.496
Epoch: 0, Batch:  1100, Loss: 3.488
Epoch: 0, Batch:  1200, Loss: 2.568
Epoch: 0, Batch:  1300, Loss: 3.263
Epoch: 0, Batch:  1400, Loss: 3.272
Epoch: 0, Batch:  1500, Loss: 3.643
Epoch: 0, Batch:  1600, Loss: 3.293
Epoch: 0, Batch:  1700, Loss: 3.306
Epoch: 0, Batch:  1800, Loss: 3.477
Epoch: 0, Batch:  1900, Loss: 3.455
Epoch: 0, Batch:  2000, Loss: 2.787
Epoch: 0, Batch:  2100, Loss: 2.725
Epoch: 0, Batch:  2200, Loss: 2.706
Epoch: 0, Batch:  2300, Loss: 2.683
Epoch: 0, Batch:  2400, Loss: 3.280
Epoch: 0, Batch:  2500, Loss: 3.227
Epoch: 0, Batch:  2600, Loss: 2.175
Epoch: 0, Batch:  2700, Loss: 2.737
Epoch: 0, Batch:  2800, Loss